Setup and data import to MySQL

In [ ]:
# 0. Install packages if needed (uncomment and run once)
# !pip install mysql-connector-python SQLAlchemy pymysql pandas matplotlib seaborn

import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
# 1. Load Excel/CSV data into pandas

# If your file is Excel:
# df = pd.read_excel("ocd_patient_dataset.xlsx")

# For your attached CSV:
df = pd.read_csv("ocd_patient_dataset.csv")

df.head()
# 2. Create MySQL connection (edit with your credentials)
user = "root"
password = "your_password"
host = "localhost"
port = 3306
database = "health_data"

connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

# 3. Load dataframe into MySQL as `ocd_patient_dataset`
table_name = "ocd_patient_dataset"

# if_exists="replace" for first time; use "append" later
df.to_sql(table_name, engine, if_exists="replace", index=False)

print("Data loaded into MySQL table:", table_name)


1. Gender counts, percentages, and average obsession score

In [ ]:
# Using pandas to replicate the CTE + aggregation logic

# Aggregation at gender level
gender_agg = (
    df.groupby("Gender")
    .agg(
        patient_count=("Patient ID", "count"),
        avg_obs_score=("Y-BOCS Score (Obsessions)", "mean")
    )
    .reset_index()
)

gender_agg["avg_obs_score"] = gender_agg["avg_obs_score"].round(2)

# Total patients for percentage
total_patients = gender_agg["patient_count"].sum()

# Compute percentage by gender
gender_agg["pct"] = (gender_agg["patient_count"] / total_patients * 100).round(2)

gender_agg
# Bar plot: number of patients by gender
plt.figure(figsize=(6,4))
sns.barplot(data=gender_agg, x="Gender", y="patient_count", palette="Set2")
plt.title("Count of Patients by Gender")
plt.ylabel("Patient Count")
plt.show()

# Bar plot: average obsession score by gender
plt.figure(figsize=(6,4))
sns.barplot(data=gender_agg, x="Gender", y="avg_obs_score", palette="Set1")
plt.title("Average Obsession Score by Gender")
plt.ylabel("Average Y-BOCS Obsession Score")
plt.show()


2. Patients by ethnicity and average obsession score

In [ ]:
ethnicity_agg = (
    df.groupby("Ethnicity")
    .agg(
        patient_count=("Patient ID", "count"),
        obs_score=("Y-BOCS Score (Obsessions)", "mean")
    )
    .reset_index()
)

ethnicity_agg["obs_score"] = ethnicity_agg["obs_score"].round(2)
ethnicity_agg.sort_values("patient_count", ascending=False, inplace=True)
ethnicity_agg
plt.figure(figsize=(8,4))
sns.barplot(
    data=ethnicity_agg,
    x="Ethnicity",
    y="patient_count",
    palette="viridis"
)
plt.title("Patients by Ethnicity")
plt.ylabel("Patient Count")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(8,4))
sns.barplot(
    data=ethnicity_agg,
    x="Ethnicity",
    y="obs_score",
    palette="magma"
)
plt.title("Average Obsession Score by Ethnicity")
plt.ylabel("Average Y-BOCS Obsession Score")
plt.xticks(rotation=45)
plt.show()


3. Number of people diagnosed with OCD month‑over‑month

In [ ]:
# Ensure date column is datetime
df["OCD Diagnosis Date"] = pd.to_datetime(df["OCD Diagnosis Date"])

# Truncate to first day of month (similar to DATE_FORMAT to 'YYYY-MM-01')
df["Diagnosis_Month"] = df["OCD Diagnosis Date"].dt.to_period("M").dt.to_timestamp()

mom_counts = (
    df.groupby("Diagnosis_Month")["Patient ID"]
    .count()
    .reset_index(name="patient_count")
    .sort_values("Diagnosis_Month")
)

mom_counts
plt.figure(figsize=(10,4))
sns.lineplot(
    data=mom_counts,
    x="Diagnosis_Month",
    y="patient_count",
    marker="o"
)
plt.title("Number of OCD Diagnoses per Month")
plt.xlabel("Month")
plt.ylabel("Patient Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

4. Most common obsession type and its average obsession score

In [ ]:
obsession_agg = (
    df.groupby("Obsession Type")
    .agg(
        patient_count=("Patient ID", "count"),
        obs_score=("Y-BOCS Score (Obsessions)", "mean")
    )
    .reset_index()
)

obsession_agg["obs_score"] = obsession_agg["obs_score"].round(2)
obsession_agg.sort_values("patient_count", ascending=False, inplace=True)
obsession_agg
# Most common obsession type
top_obsession = obsession_agg.iloc[0]
top_obsession
plt.figure(figsize=(8,4))
sns.barplot(
    data=obsession_agg,
    x="Obsession Type",
    y="patient_count",
    palette="coolwarm"
)
plt.title("Patients by Obsession Type")
plt.ylabel("Patient Count")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(8,4))
sns.barplot(
    data=obsession_agg,
    x="Obsession Type",
    y="obs_score",
    palette="Blues"
)
plt.title("Average Obsession Score by Obsession Type")
plt.ylabel("Average Y-BOCS Obsession Score")
plt.xticks(rotation=45)
plt.show()


5. Most common compulsion type and its average obsession score

In [ ]:
compulsion_agg = (
    df.groupby("Compulsion Type")
    .agg(
        patient_count=("Patient ID", "count"),
        obs_score=("Y-BOCS Score (Obsessions)", "mean")
    )
    .reset_index()
)

compulsion_agg["obs_score"] = compulsion_agg["obs_score"].round(2)
compulsion_agg.sort_values("patient_count", ascending=False, inplace=True)
compulsion_agg
# Most common compulsion type
top_compulsion = compulsion_agg.iloc[0]
top_compulsion
plt.figure(figsize=(8,4))
sns.barplot(
    data=compulsion_agg,
    x="Compulsion Type",
    y="patient_count",
    palette="GnBu"
)
plt.title("Patients by Compulsion Type")
plt.ylabel("Patient Count")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(8,4))
sns.barplot(
    data=compulsion_agg,
    x="Compulsion Type",
    y="obs_score",
    palette="OrRd"
)
plt.title("Average Obsession Score by Compulsion Type")
plt.ylabel("Average Y-BOCS Obsession Score")
plt.xticks(rotation=45)
plt.show()
